In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 5 - similar to example_forkJoin_1 but closed
model = Network('model')

In [3]:
# Create network
delay = Delay(model,'Delay')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
fork = Fork(model, 'Fork')
join = Join(model, 'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 5, delay, 0)

In [4]:
# Service configurations
delay.set_service(jobclass1, Exp(1.0))
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))

In [5]:
# Set routing matrix
P = model.init_routing_matrix()
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)
model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000)]
# Add MVA with specific options like MATLAB
options = SolverMVA.defaultOptions()
options.config.fork_join = 'ht'
options.method = 'amva'
solvers.append(SolverMVA(model, options))

for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/16736691655599200892/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.014385s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   class1  0.8791  0.8791  0.9950  0.9950  0.8801  0.8764
1  Queue1   class1  2.6764  0.8854  3.0212  1.5106  0.8764  0.8715
2  Queue2   class1  2.6266  0.8860  3.0353  1.5177  0.8764  0.8728
3    Join   class1  2.9037  0.0000  1.6797  1.6797  1.7657  0.8801
SOLVER 2: SolverMVA
MVA analysis [method: default(exact), lang: java, env: 21.0.8] completed in 0.051491s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   class1  0.7669  0.7669  1.0000  1.0000  0.7669  0.7669
1  Queue1   class1  1.4521  0.7669  1.8936  0.9468  0.7669  0.7669
2  Queue2   class1  3.8343  0.7669  5.0000  2.5000  0.7669  0.7669
3    Join   class1  3.1799  0.0000  2.0734  2.0734  1.5337  0.7669
